# SAS On-the-Job | Part 1 
## Role: Data Analyst within the Department of Health and Human Services

#### Starting with a Story
The coronavirus outbreak, or COVID-19, simultaneously created a health and economic crisis in the United States. As local regions locked down to slow the spread of the disease, millions of jobs were lost across the United States. Most schools also shuttered their doors and switched to remote learning. This meant that many working parents who retained their jobs found themselves as both remote workers and teacher’s assistants during the lockdowns. <br>

The impacts of COVID-19 have been very uneven, especially for mothers. A focus of public policy for decades has been to find a way to get mothers back into the labor force, while supporting them as they balance work and home responsibilities. In this SAS On-the-Job activity, we explore the pre- and post-COVID-19 trends in unemployment (UE) and labor force participation (LFP) rates among U.S. women aged 25 to 54. <br>

#### Job Setting 
You are a new *Public Policy Analyst* within the Department of Health and Human Services (HHS). You are placed into a department interested in public policies that promote labor force participation (LFP) by women – particularly low-income and less-educated women. Labor force participation simply means that women are engaged in the labor market - either they have a job or are looking for one.  We'll also explore the more familiar unemployment rate, which captures women who would like a job, but don't currently have one.<br>

During the height of the pandemic, a host of media articles highlighted the disproportionate impact of COVID-19 on women’s employment.  HHS leadership is interested in knowing whether these impacts are still being felt today - years after the official declaration of the pandemic.  If yes, HHS leadership would consider enacting new, target policy response to support this potentially vulnerable group of workers. However, before considering policy responses, HHS leadership would like someone to examine whether there are lingering effects of COVID-19 on current level of labor supply by women. And this someone is you!  Drumroll, please!<br>

As a new analyst, your goals in this set of exercises are to<br>
•	access the Jupyter notebooks created by your predecessor <br>
•	understand what the previous analyst did - particularly with the integration of SAS, SQL, Python, and R <br>
•	modify existing code (as appropriate). <br>

And you'll be guided in this exercise by a very helpful onboarding buddy. Which is me. Your humble narrator.

Let's get started!

#### From a technical perspective, this SAS On-the-Job provides
Beyond allowing you to play policy analyst for the day, this SAS On-the-Job (OTJ) provides an overview of SAS, SQL, Python, and R integration in Jupyter.  More specifically, this SAS OTJ is an act in three parts, with the following flow: <br>

(1) Start in a Python notebook - because the original coder loved Python.  Perform a preliminary analysis of the data.  Then use SQL to collapse the data - because it's a much more effective way to aggregate data than other approaches in Python.  And finally, we'll ensure that the underlying data make sense. <br>
(2) The second Jupyter notebook focuses on SAS code. We'll leverage some SAS code and macros to explore aggregated U.S. trends. We'll conclude by creating a state-level data set that we can then use to map trends in R (confession: R can produce some great maps!). <br>
(3) The last notebook focuses on R code.  We'll plot state-level trends over time for unemployment and labor force participation. <br>

## Python, SAS and SQL Integration
### American Community Survey (ACS) Data Import 

Welcome to Day 1 as Public Policy Analyst at HHS. In your exploration of labor supply by women during COVID, the first thing we need to do is to get our data loaded into the environment and running. Toward that goal, let's first connect with the SAS Viya server - where we will run our entire analysis. After making this connection, we'll then use the "SASPy" Python package to establish a connection between SAS and Python - and run some SAS code within the Python kernel to load the American Community Survey Data. <br> 

And why start with the SAS code?  Well, this project has used hackers from all backgrounds in the analysis - and the previous analyst was a big fan of using SAS to read in their data. But, know that reading in the data via Python would have been perfectly acceptable too!

In [ ]:
import saspy

# Create a SAS session object
sas_session = saspy.SASsession()

With the connection established, we can now use the [`submit` method](https://sassoftware.github.io/saspy/api.html#saspy.SASsession.submit) to run SAS code from Jupyter using a Python kernel. Notice how we use the LIBNAME statement to save the ACS file to disk - and while pulling the original data from GitHub. Moreover, we create a Python data frame at the end, so that we can forge on with the Python portion of the analysis.

In [ ]:
results_dict = sas_session.submitLST(
             """
                libname acs "/home/student/S23HSRP";
                
                filename acs_url url 'https://raw.githubusercontent.com/lincolngroves/SAS-OTJ-HHS/main/ACS_2015_2022_ltd.csv';
                
                   data acs.acs_2015_2022    ;
                        %let _EFIERR_ = 0; /* set the ERROR detection macro variable */

                        infile acs_url delimiter = ',' MISSOVER DSD  firstobs=2 ;

                        informat State_FIP best32. ;
                        informat State_Name $20. ;
                        informat YearQuarter YYQ6. ;
                        informat Race_Ethnic $21. ;
                        informat EDUC_LTD $19. ;
                        informat Child_Status $14. ;
                        informat Unemp best32. ;
                        informat in_LF best32. ;
                        informat WTFINL best32. ;

                        format State_FIP best12. ;
                        format State_Name $20. ;
                        format YearQuarter YYQ6. ;
                        format Race_Ethnic $21. ;
                        format EDUC_LTD $19. ;
                        format Child_Status $14. ;
                        format Unemp best12. ;
                        format in_LF best12. ;
                        format WTFINL best12. ;
                        
                        input
                            State_FIP
                            State_Name  $
                            YearQuarter
                            Race_Ethnic  $
                            EDUC_LTD  $
                            Child_Status  $
                            Unemp
                            in_LF
                            WTFINL
                        ;
                        if _ERROR_ then call symputx('_EFIERR_',1);  /* set ERROR detection macro variable */

                        run;
              """,
             )

acs_df = sas_session.sd2df(table="acs_2015_2022",libref="acs")

## Examine and Explore Our Data Frame

Alright - the data are now accessible in the notebook.  What's typically next as an analyst?  Well, let's examine the underlying data to become more familiar with the data and distribution of values!

In [ ]:
# One Simple line will pull up the Python dataframe
acs_df

From the data above, we can see that we'll have nine variables in our analysis.  The current unit of analysis in this data is state, year-quarter, race-ethnic, and child-status.  In simple English, this means that a row - that is, the observation - contains the unemployment rate and labor force participation rate for women of a particular race-ethnic group, education level, and child status - for a given state at a point in time.  Yup, that's a mouthful, but still very important.  Moreover, note that **Unemp** and **in_LF** are rounded in the window above - but are not in the underlying data.  Finally, **WTFINL** is a weighting variable that represents the number of women represented in that row.  So, 8709.3981 can be simply interpreted as ~8709 women. This variable will be very important when we aggregate the data.

Finally, **FIPS** are Federal Information Processing Standards, so the **State_Fip** is just a unique variable for that state. And it will help in the upcoming merges. Finally, **State** is just, well, a U.S. state.

With data now upload in Python, let's load some Python packages so that we can do some cooler things with the data!

In [ ]:
# Classic Python packages that will make our Exploratory Data Analysis easier 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Summarize the Data

Let's summarize the data in Python - so that we can better understand the distribution of the numeric variables. As a wise person might or might not have stated: *gotta know thy data!*

In [ ]:
# Perform the PROC MEANS equivalent in Python
summary = acs_df.describe()
summary = summary.round(2)  # Set the decimal places to 2

# Print the summary statistics
print(summary)

We can see from *Unemp* and *in_LF* above that we do have a distribution of values - and that the average (unweighted) unemployment rate over this period is 18% and the average (unweighted) labor force participation rate is 62%.  Ideally, we'd want to weight these values statistically to get a true U.S. average - but this is a fine sanity check for now. The primary take away is that the data appear valid and - fun fact - there are no missing values. 

How do we know the latter?  Well, the count is the same across the four variables - and it also matches the number of variables in the data set. (See log above.)

## Understand Distribution of Select Variables

Data appear to be valid and clean.  What a great way to start as an analyst... because - truth be told - this never happens.

Our next task is to examine the distribution of the categorical variables, via both tables and charts.  For example, it would be useful to know how *Education*, *Child Status*, and so on, are distributed in the data.

Let's use some classic Python procedures to do exactly that.  We'll keep the analysis unweighted, which still allows us to examine the relative proportion of each variable in the analysis. This is perfectly fine test for this stage of the analysis. :)

In [ ]:
# Set the title for the frequency analysis
title = "Frequencies for Categorical Variables"

# Perform frequency analysis
categorical_variables = ['Race_Ethnic', 'EDUC_LTD', 'Child_Status', 'Unemp', 'in_LF']
for variable in categorical_variables:
    freq_table = acs_df[variable].value_counts().reset_index()
    freq_table.columns = ['Value', 'Frequency']
    
    # Print the frequency table
    print(f"\n{title}")
    print(f"\n{variable}")
    print(freq_table)
    
    # Create frequency plot
    sns.countplot(data=acs_df, x=variable)
    plt.title(f"{title}\n{variable}")
    plt.show()

Do you see any interesting trends in the underlying data?  Again, these data aren't weighted, so they don't represent U.S. averages. But they're a useful statistic nonetheless.

My interesting findings: (1) White, Non-Hispanic are in, by far, the most states across the United States. (2) There are several states in the U.S. that we couldn't calculate estimates for individuals with less than a high school level of education, because sample sizes were either too small or not available. (3) Most women in the data set (a) had children, (b) were in the labor force, and (c) were employed.

## Use SQL Code to Collapse Data to Get U.S. Estimates

As noted above, an observation in the data is a particular demographic group residing in a particular state in a given year. Because we have the weighed value for each cell,    we can use SQL to aggregate the data to the country level and give us a more precise U.S. average over time. Ready to see the how elegantly SQL can collapse data?  Well, I am!

In [ ]:
results_dict = sas_session.submit(
    """
       libname acs "/home/student/S23HSRP";
        
        proc sql;
            create 	table acs.covid_labor_supply_us as 
            select	distinct 
                    yearquarter , 

        /*******************************************************************  Labor Force Status | All  */
                    sum( ( unemp=1 ) * WTFINL ) 											/ sum( ( in_LF=1 ) *   WTFINL )											as UE_Women				label="Unemployment Rate"	format percent9.1 		,
                    sum( ( in_LF=1 ) * WTFINL ) 											/ sum(  			   WTFINL ) 										as LFP_Women			label="LFP Rate"			format percent9.1 		,


        /*******************************************************************  Labor Force Status | By Race  */

                    /*******************************************************  Unemployment */
                    sum( ( race_ethnic="Black, Non-Hispanic" ) * ( unemp=1 ) * WTFINL ) 	/ sum( ( race_ethnic="Black, Non-Hispanic" ) * ( in_LF=1 ) * WTFINL ) 	as UE_BlackWomen		label="Black Women" 	format percent9.1 		,
                    sum( ( race_ethnic="Hispanic" ) * ( unemp=1 ) * WTFINL ) 				/ sum( ( race_ethnic="Hispanic" ) * ( in_LF=1 ) * WTFINL ) 				as UE_HispanicWomen		label="Hispanic Women" 	format percent9.1 		,
                    sum( ( race_ethnic="White, Non-Hispanic" ) * ( unemp=1 ) * WTFINL ) 	/ sum( ( race_ethnic="White, Non-Hispanic" ) * ( in_LF=1 ) * WTFINL ) 	as UE_WhiteWomen		label="White Women" 	format percent9.1 		,
                    sum( ( race_ethnic="All Other" ) * ( unemp=1 ) * WTFINL ) 				/ sum( ( race_ethnic="All Other" ) * ( in_LF=1 ) * WTFINL ) 			as UE_OtherWomen		label="All Other Women" format percent9.1 		,

                    /*******************************************************  LFP */
                    sum( ( race_ethnic="Black, Non-Hispanic" ) * ( in_LF=1 ) * WTFINL ) 	/ sum( ( race_ethnic="Black, Non-Hispanic" ) * WTFINL )					as LFP_BlackWomen		label="Black Women" 	format percent9.1 		,
                    sum( ( race_ethnic="Hispanic" ) * ( in_LF=1 ) * WTFINL ) 				/ sum( ( race_ethnic="Hispanic" ) * WTFINL ) 							as LFP_HispanicWomen	label="Hispanic Women" 	format percent9.1 		,
                    sum( ( race_ethnic="White, Non-Hispanic" ) * ( in_LF=1 ) * WTFINL ) 	/ sum( ( race_ethnic="White, Non-Hispanic" ) * WTFINL ) 				as LFP_WhiteWomen		label="White Women" 	format percent9.1 		,
                    sum( ( race_ethnic="All Other" ) * ( in_LF=1 ) * WTFINL ) 				/ sum( ( race_ethnic="All Other" ) * WTFINL )							as LFP_OtherWomen		label="All Other Women" format percent9.1 		,


        /*******************************************************************  Labor Force Status | By Education  */

                    /*******************************************************  Unemployment */
                    sum( ( educ_ltd="< HS" ) * ( unemp=1 ) * WTFINL ) 						/ sum( ( educ_ltd="< HS" ) * ( in_LF=1 ) * WTFINL )						as UE_Women_LTHS		label="EDUC < HS" 		format percent9.1 		,
                    sum( ( educ_ltd="High School Diploma" ) * ( unemp=1 ) * WTFINL ) 		/ sum( ( educ_ltd="High School Diploma" ) * ( in_LF=1 ) * WTFINL ) 		as UE_Women_HS			label="EDUC = HS" 		format percent9.1 		,
                    sum( ( educ_ltd="Some College" ) * ( unemp=1 ) * WTFINL ) 				/ sum( ( educ_ltd="Some College" ) * ( in_LF=1 ) * WTFINL ) 			as UE_Women_SCollege	label="Some College"	format percent9.1 		,
                    sum( ( educ_ltd="College +" ) * ( unemp=1 ) * WTFINL ) 					/ sum( ( educ_ltd="College +" ) * ( in_LF=1 ) * WTFINL ) 				as UE_Women_CollegeP	label="College +" 		format percent9.1 		,

                    /*******************************************************  LFP */
                    sum( ( educ_ltd="< HS" ) * ( in_LF=1 ) * WTFINL ) 						/ sum( ( educ_ltd="< HS" ) * WTFINL ) 									as LFP_Women_LTHS		label="EDUC < HS" 		format percent9.1 		,
                    sum( ( educ_ltd="High School Diploma" ) * ( in_LF=1 ) * WTFINL ) 		/ sum( ( educ_ltd="High School Diploma" ) * WTFINL ) 					as LFP_Women_HS			label="EDUC = HS" 		format percent9.1 		,
                    sum( ( educ_ltd="Some College" ) * ( in_LF=1 ) * WTFINL ) 				/ sum( ( educ_ltd="Some College" ) * WTFINL ) 							as LFP_Women_SCollege	label="Some College" 	format percent9.1 		,
                    sum( ( educ_ltd="College +" ) * ( in_LF=1 ) * WTFINL ) 					/ sum( ( educ_ltd="College +" ) * WTFINL ) 								as LFP_Women_CollegeP	label="College +" 		format percent9.1 		,


        /*******************************************************************  Labor Force Status | By Child Status  */

                    /*******************************************************  Unemployment */
                    sum( ( child_status="No Children" ) * ( unemp=1 ) * WTFINL ) 			/ sum( ( child_status="No Children" ) * ( in_LF=1 ) * WTFINL )			as UE_Women_NoKids		label="No Children" 	format percent9.1 		,
                    sum( ( child_status="Older Children" ) * ( unemp=1 ) * WTFINL ) 		/ sum( ( child_status="Older Children" ) * ( in_LF=1 ) * WTFINL ) 		as UE_Women_OlderKids	label="Older Children" 	format percent9.1 		,
                    sum( ( child_status="Child < 5" ) * ( unemp=1 ) * WTFINL ) 				/ sum( ( child_status="Child < 5" ) * ( in_LF=1 ) * WTFINL ) 			as UE_Women_YoungKids	label="Young Children"	format percent9.1 		,

                    /*******************************************************  LFP */
                    sum( ( child_status="No Children" ) * ( in_LF=1 ) * WTFINL ) 			/ sum( ( child_status="No Children" ) * WTFINL ) 						as LFP_Women_NoKids		label="No Children" 	format percent9.1 		,
                    sum( ( child_status="Older Children" ) * ( in_LF=1 ) * WTFINL ) 		/ sum( ( child_status="Older Children" ) * WTFINL ) 					as LFP_Women_OlderKids	label="Older Children" 	format percent9.1 		,
                    sum( ( child_status="Child < 5" ) * ( in_LF=1 ) * WTFINL ) 				/ sum( ( child_status="Child < 5" ) * WTFINL ) 							as LFP_Women_YoungKids	label="Young Children"	format percent9.1 		


            from 	acs.acs_2015_2022
            group	by 1 
            order	by 1 ;
        quit;

    """, 
)
    
covid_labor_supply_us_df = sas_session.sd2df(table="covid_labor_supply_us",libref="acs")

Lots of good stuff in that statement above. Note that we're collapsing to the yearquarter level - and incorporating the sample weight (that is, WTFINL) in the analysis.  Notice how succinct that code is - which would take many traditional Python or SAS commands if we went that route instead. <cr>
    
Let's now summarize the data one more time, to ensure that we aggregated the data properly. One bad sign would be to see unemployment rate and labor force participation rates either less than 0 or greater than 1.  Because those things shouldn't happen.

### Sanity Check: Do the Data Make Sense?
Let's print the data just to ensure that our data seem reasonable. The Python code:

In [ ]:
#  Print the Data
covid_labor_supply_us_df

Let's further probe the data:

In [ ]:
# Summarize the Data
summary_us = covid_labor_supply_us_df.describe()
summary_us = summary_us.round(3)  # Set the decimal places to 3

# Print the summary statistics
print(summary_us)

What do you see?  My musings: (1) there are 32 observations in the aggregated data set.  The math: 8 years * 4 quarters.  (2) The (weighted) average unemployment rate is 0.044 - or 4.4%.  This value is much lower than the 0.18 (18%) value reported above - which shows that weighting is consequential and likely higher in the smaller states. (3) We see that, in general, the unemployment rate decreases, and the labor force participation rate increases, as the level of education increases. All these nuggets make sense!

### Part 1 Recap

Our project is off to a great start.  We've imported the data, performed some preliminary data analysis using Python code, and collapsed the data using SQL.  But, we really haven't answered any of the interesting questions for HHS Leadership: namely (1) what are the UE and LFP trends over time and (2) are the levels back to "normal" after a COVID adjustment period? 

We'll tackle these questions better in Parts 2 and 3 of this analysis.